To pair satellite images with PM and Met, this script requires:
- CSV with location names and coordinates 
- Folder with all locations' PM in seperate .xls files. .xls files should be named after location
- CSV with meteorology

In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals


# Common imports
import numpy as np
import pandas as pd
import os
# to make this notebook's output stable across runs
np.random.seed(42)

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import rasterio
from rasterio.merge import merge
from rasterio.plot import show

import json
from shapely.geometry import shape
from shapely.ops import transform as stransform
from rasterio.mask import mask
from affine import Affine
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import pyproj
import pickle as pkl
import geojson
import cv2

from rasterio.warp import transform as rtransform
from rasterio.transform import rowcol

import PIL
from PIL import Image, ImageEnhance
import sys
# from skimage import io

os.getcwd()

'/Users/sarah/Satellite_Image_Processing'

In [2]:
# #coordinates
my_coordinates = pd.read_csv('../Desktop/DHAKA.csv')
my_coordinates

FileNotFoundError: [Errno 2] No such file or directory: '../Desktop/DHAKA.csv'

In [5]:
PM25_dict = {}
sensor_location = '../Downloads/dhaka.xls'

def pm_df(my_coordinates, path):
    site_name = path.split('/')[-1].split('.xls')[0]
    site_coordinate = (my_coordinates[my_coordinates.SITE == site_name].values[0,1],\
                    my_coordinates[my_coordinates.SITE == site_name].values[0,2])
    print(site_coordinate)
    df = pd.read_excel(path)
    df['DATE'] = pd.to_datetime(df['DATE'],dayfirst=True) 
    df = df.set_index(pd.DatetimeIndex(df['DATE']))
    df_PM25 = df['PM25'].to_frame().dropna()
#     print(df)
            
    if len(df_PM25)>0:
        PM25_dict[site_name] = {'PM25':df_PM25, 'coordinate':site_coordinate}
    return PM25_dict

# except Exception as e:
#     print(e.args)

In [6]:
PM25_dict = pm_df(my_coordinates, sensor_location)

(23.796725, 90.42201999999999)


In [7]:
PM25_dict

{'dhaka': {'PM25':             PM25
  DATE            
  2017-01-01   249
  2017-01-02   234
  2017-01-03   165
  2017-01-04   152
  2017-01-05   224
  ...          ...
  2021-12-27   195
  2021-12-28   193
  2021-12-29    82
  2021-12-30   136
  2021-12-31   103
  
  [1680 rows x 1 columns],
  'coordinate': (23.796725, 90.42201999999999)}}

## Meteorology

In [6]:
# def met_dict(metpath):
#     meteorology_raw_concat_no_NA_daily_avg_final = pd.read_csv(metpath)
#     meteorology_raw_concat_no_NA_daily_avg_final['DATE'] = pd.to_datetime(meteorology_raw_concat_no_NA_daily_avg_final['DATE'])
#     meteorology_raw_concat_no_NA_daily_avg_final = meteorology_raw_concat_no_NA_daily_avg_final.set_index(pd.DatetimeIndex(meteorology_raw_concat_no_NA_daily_avg_final['DATE']))

#     meteorology_raw_concat_no_NA_daily_avg_final.drop(['DATE'], axis = 1, inplace = True)

#     return meteorology_raw_concat_no_NA_daily_avg_final

In [8]:
# meteorology_raw_concat_no_NA_daily_avg_final= met_dict('../Downloads/dhaka_met.csv')
# meteorology_raw_concat_no_NA_daily_avg_final

In [9]:
# def PM_Meteo_Matcher(Processed_PM_dfs_concat, coords_AQM):
#     #load PM2.5
#     Processed_PM_dfs_concat = Processed_PM_dfs_concat.apply(lambda x: pd.to_numeric(x))
    
#     cur_lat, cur_long = coords_AQM 
# #     print(Processed_PM_dfs_concat, cur_lat, cur_long)
#     #sort meteorology stations based on their distances to the AQM station
#     #sort default in ascending order
#     #meteo_stations_closest_in_order = [station_id for station_id ,_, _ in \
#     #        sorted(zip(meteo_station_IDs, meteo_station_lats, meteo_station_longs), \
#     #               key=lambda triplet: geopy.distance.vincenty(coords_AQM, (triplet[1], triplet[2])).km)]
    
#     #Matching meteo to PM2.5
#     Processed_PM_dfs_concat_meteo = pd.merge(Processed_PM_dfs_concat, meteorology_raw_concat_no_NA_daily_avg_final,\
#                                              left_index=True,right_index=True,how='inner').resample('1d').mean().dropna()
#     '''
#     for station_id in meteo_stations_closest_in_order[1:]:
#         meteo_missing_index  = Processed_PM_dfs_concat_meteo[Processed_PM_dfs_concat_meteo.iloc[:,1:].isna().any(1)].index
#         if len(meteo_missing_index)>0:
#             #print(meteo_missing_index)
#             Processed_PM_dfs_concat_meteo.loc[meteo_missing_index,kept_features] = \
#             Processed_PM_dfs_concat_meteo.loc[meteo_missing_index,kept_features].fillna(LA_meteo[station_id].loc[meteo_missing_index,kept_features])
#         else:
#             break
#     '''
#     #Processed_PM_dfs_concat_meteo.dropna(inplace = True)
#     return Processed_PM_dfs_concat_meteo

In [9]:
# PM25_meteo_matched = {key:PM_Meteo_Matcher(PM25_dict[key]['PM25'], PM25_dict[key]['coordinate']) for key in PM25_dict.keys()}

In [10]:
# PM25_meteo_matched

In [36]:
final_image_path = 'Documents/dhaka/20211220_050630_1061_3B_Visual_clip.tif'
station_index = final_image_path.split('/')[-2]
time_index = final_image_path.split('/')[-1].split('_')[1]
time_index = time_index[0:2]+':'+time_index[2:4]+':' +time_index[4:]
id_index = final_image_path.split('/')[-1].split('_')[2]
id_index
date_index = final_image_path.split('/')[-1].split('_')[0]

date_index = date_index[0:4] + '-' + date_index[4:6] + '-' + date_index[6:]
date_index

'2021-12-20'

In [48]:
PM25_dict[station_index]['PM25']['PM25'][pd.Timestamp(2017,1,5)]

224

In [31]:
date_index = [pd.Timestamp(2017,1,1), pd.Timestamp(2017,1,3), pd.Timestamp(2017,1,5)]

In [11]:
image_paths = [join('../Documents/dhaka', f) for f in sorted(listdir('../Documents/dhaka')) if isfile(join('../Documents/dhaka', f)) and '.DS_Store' not in str(f)]

In [14]:
result = []
for path in image_paths:
    im = np.moveaxis(rasterio.open(path).read(), 0, 2)
    station_index = path.split('/')[-2]
    time_index = path.split('/')[-1].split('_')[1]
    time_index = time_index[0:2]+':'+time_index[2:4]+':' +time_index[4:]
    date_index = path.split('/')[-1].split('_')[0]
    for date in PM25_dict[station_index]['PM25']['PM25'].index:
        yr, mo, dt = int(date_index[:4]), int(date_index[4:6]), int(date_index[6:])
        if date == pd.Timestamp(yr, mo, dt):
            im_pm_pair = {}
            im_pm_pair['Image'] = im
            im_pm_pair['PM'] = PM25_dict[station_index]['PM25']['PM25'][date]
            result.append(im_pm_pair)

/Users/sarah/opt/anaconda3/lib/python3.8/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [54]:
result

[{'Image': array([[[108, 117, 110, 255],
          [104, 113, 107, 255],
          [106, 113, 108, 255],
          ...,
          [175, 167, 152, 255],
          [174, 166, 150, 255],
          [173, 165, 149, 255]],
  
         [[116, 126, 117, 255],
          [106, 115, 108, 255],
          [112, 118, 114, 255],
          ...,
          [178, 170, 155, 255],
          [177, 170, 154, 255],
          [172, 166, 150, 255]],
  
         [[119, 127, 118, 255],
          [108, 116, 109, 255],
          [108, 116, 112, 255],
          ...,
          [183, 175, 159, 255],
          [180, 172, 156, 255],
          [171, 165, 149, 255]],
  
         ...,
  
         [[180, 173, 162, 255],
          [202, 194, 180, 255],
          [198, 191, 173, 255],
          ...,
          [121, 125, 119, 255],
          [125, 125, 118, 255],
          [148, 145, 134, 255]],
  
         [[166, 160, 149, 255],
          [212, 202, 188, 255],
          [204, 196, 181, 255],
          ...,
          [122, 126

In [10]:
def Imagery_matcher(final_image_path):
    try:
        print('site loading')
        im = np.moveaxis(rasterio.open(final_image_path).read(), 0, 2)
        img_arr = np.array(im)
#         print('read in working')
    except:
        print(final_image_path+' cannot be loaded')
        return None
    
    black_space = np.mean(img_arr[:,:,:]/255)

#     if black_space > 70 or black_space <= 10:
#         return None
    if black_space > .91 or black_space <= 0.4:
#         print(black_space)
#         plt.imshow(im)
#         plt.show()
        return None
    else:
        
        station_index = final_image_path.split('/')[2]
        time_index = final_image_path.split('/')[-1].split('_')[1]
        time_index = time_index[0:2]+':'+time_index[2:4]+':' +time_index[4:]
        date_index = final_image_path.split('/')[-1].split('_')[0]
        date_index = date_index[0:4] + '-' + date_index[4:6] + '-' + date_index[6:]
        id_index = final_image_path.split('/')[-1].split('_')[2]
#         print('indexes working')
#         plt.imshow(im)
#         plt.show()

    try:
        matching_PM25 = PM25_dict[station_index].index == date_index
#         matching_PM25 = PM25_meteo_matched[station_index][PM25_meteo_matched[station_index].index == date_index].iloc[0,0]
        print('matched pm working')
        
#         matching_meteo = PM25_meteo_matched[station_index][PM25_meteo_matched[station_index].index == date_index].iloc[0,1:]
#         print('matched met working')
        
    except Exception as e:
        print('above failed')
        print(e.args)
        return None

    return({'Image':im, 'PM':matching_PM25, 'Meteo':matching_meteo, 'Date':date_index, 'Time_stamp':time_index,
            'Station_index': station_index, 'Sat_ID':id_index})




In [11]:
def Image_loader(image_folder_path):
    my_current_image_path = image_folder_path
    print(my_current_image_path)
    Matching_data_for_a_single_station = [Imagery_matcher(join(my_current_image_path, f)) for f in sorted(listdir(my_current_image_path)) \
                            if isfile(join(my_current_image_path, f)) and '.DS_Store' not in str(f)\
                                         and Imagery_matcher(join(my_current_image_path, f)) is not None]
    return Matching_data_for_a_single_station

In [14]:
AQM_data_for_model = Image_loader('../Documents/dhaka') #+ site) for site in my_coordinates.SITE]

../Documents/dhaka
site loading
above failed
('single positional indexer is out-of-bounds',)
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm

matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm w

matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading

matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm w

matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm w

...

matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
above failed
('single positional indexer is out-of-bounds',)
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
sit

matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met 

matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm w

matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm w

matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm working
matched met working
site loading
matched pm w

In [85]:
# for i in range(len(AQM_data_for_model)):
#     h = len(AQM_data_for_model[i])
#     print(np.sum(h))

In [15]:
with open("Desktop/K_224_met_aod_1721.pkl", "wb") as fp:
    pkl.dump(AQM_data_for_model, fp)

In [15]:
AQM_data_for_model[0]

{'Image': array([[[108, 117, 110, 255],
         [104, 113, 107, 255],
         [106, 113, 108, 255],
         ...,
         [175, 167, 152, 255],
         [174, 166, 150, 255],
         [173, 165, 149, 255]],
 
        [[116, 126, 117, 255],
         [106, 115, 108, 255],
         [112, 118, 114, 255],
         ...,
         [178, 170, 155, 255],
         [177, 170, 154, 255],
         [172, 166, 150, 255]],
 
        [[119, 127, 118, 255],
         [108, 116, 109, 255],
         [108, 116, 112, 255],
         ...,
         [183, 175, 159, 255],
         [180, 172, 156, 255],
         [171, 165, 149, 255]],
 
        ...,
 
        [[180, 173, 162, 255],
         [202, 194, 180, 255],
         [198, 191, 173, 255],
         ...,
         [121, 125, 119, 255],
         [125, 125, 118, 255],
         [148, 145, 134, 255]],
 
        [[166, 160, 149, 255],
         [212, 202, 188, 255],
         [204, 196, 181, 255],
         ...,
         [122, 126, 118, 255],
         [115, 116, 105, 2

<!-- # def Image_loader(image_folder_path):
#     my_current_image_path = image_folder_path
#     Matching_data_for_a_single_station = [image_pairing(join(my_current_image_path,f)) for f in sorted(listdir(my_current_image_path)) \
#                                       if isfile(join(my_current_image_path, f)) and '.DS_Store' not in str(f)]

#     Matching_data_for_a_single_station = list(filter(None, Matching_data_for_a_single_station))
#     return Matching_data_for_a_single_station -->

<!-- # American_club = Image_loader('Documents/planet_imgs/americanclub')
# US_embassy = Image_loader('Documents/planet_imgs/usembassy')
# budhani = Image_loader('Documents/planet_imgs/bud_imgs')
# thamel = Image_loader('Documents/planet_imgs/thamel_imgs')
# kote = Image_loader('Documents/planet_imgs/kote_imgs')
# kalank = Image_loader('Documents/planet_imgs/kalanki_imgs')
gyna = Image_loader('Documents/planet_imgs/gyaneshwore/') -->